# Read binary files generate by the modflow family

The format is the same for head files, cbc files and mt3dms conc files.

@TO 2021 10 30

Format from MODFLOW help site (under advanced, look for binary)

Binary files generated by MODFLOW are of several types.  These include:

    1	Head and Drawdown files (See CHEDFM and CDDNFM in Output Control.)
    2	Heads and flows interpolated to hydrogeologic units (See IOHUFHEADS and IOHUFFLOWS in the HUF2 package.)
    3.	Data related to subsidence or compaction (See the IBS, SUB, and SWT Packages.)
    4.	Cell-by-cell flow files (See ILPFCB in the LPF package and similar variables in other packages.)
    5.	Zeta values saved by the SWI2 package
    6.	Files saved by the SWR process. The formats for these files are not described here.

If you need to write code to read the binary files, the following descriptions of how the data is saved may help.
    hmtoggle_plus1	Array Data (#1 to #3)

The array data can be either in single-precision or double-precision format.  There is no data field that indicates which format is used.  Instead, you will need to try both and see which one works.

First read the following variables in order:

    KSTP: the time step number, an integer, 4 bytes.
    KPER: the stress period number, an integer, 4 bytes.
    PERTIM: the time in the current stress period, a real number, either 4 or 8 bytes.
    TOTIM, the total elapsed time, a real number, either 4 or 8 bytes.
    DESC, a description of the array, 16 ANSI characters, 16 bytes.
    NCOL, the number of columns in the array, an integer, 4 bytes.
    NROW, the number of rows in the array, an integer, 4 bytes.
    ILAY, the layer number, an integer, 4 bytes.
    
Next come a list of NROW x NCOL real numbers that represent the values of the array.  The values are in row major order.  Each value in the array occupies either 4 or 8 bytes depending on whether the values are in single- or double-precision.
    
After reading one set of values, start over with KSTP. Continue until reaching the end of the file.

The best way to read the binary file is using the regular python struct module.

In [114]:
import os
import struct
import numpy as np
import warnings
import pdb

In [115]:
def get_binary_fmt(fp):
    """Return format string suitable for struct module.
    
    We try both single and if it does not work doubple precision. The wrong one
    fails when converting the desc-bytes to a string.
    
    Parameters
    ----------
    fp: file pointer
        open binary file
    fmt: str
        the trial format string according to struct module with the parameters
        expected in the binary file and their byte lengths.
        
    Returns
    -------
    format: str
        complete and correct format string to be used with the struct module
    precision: str
        the single or double precision float format ('<i4' or '<i8')
    nrow: int
        number of rows
    ncol: int
        number of columns
    """
    fp.seek(0,0)
    try:
        fmt = '<iiff16siii'
        tpl = struct.unpack(fmt, fp.read(struct.calcsize(fmt)))
        kstp, kper, pertim, totim, desc, ncol, nrow, ilay = tpl # split the tuple
        desc = desc.decode('utf-8') # convert to str
        fp.seek(0, 0)
        #pdb.set_trace()
        return fmt + f'{ncol * nrow * 4:d}s', '<f4', ncol, nrow
    except:
        fmt = '<iidd16siii'
        tpl = struct.unpack(fmt, fp.read(struct.calcsize(fmt)))
        kstp, kper, pertim, totim, desc, ncol, nrow, ilay = tpl # split the tuple
        desc = desc.decode('utf-8') # convert to str
        fp.seek(0, 0)
        return fmt + f'{ncol * nrow * 8:d}s', '<f8', ncol, nrow
    else:
        raise ValueError("Can't get format for this presumed binary modflow file.")
        

def filelen(fp):
    """Return file length in bytes and rewind file."""
    L = fp.seek(0, 2)
    fp.seek(0, 0)
    return L

def recs_in_file(fp, fmt):
    """Return the total number of complete records in the file and the remainder bytes.
    
    Parameters
    ----------
    fp: file pointer
        open pointer to binary file
    fmt: str
        complete format according to struct (where array values are coded as a string,
        which will be docode later on to floats)
    """
    Lfile = filelen(fp)
    Lstr  = struct.calcsize(fmt)
    nrec = Lfile // Lstr # number of complete records
    nrem = Lfile % Lstr  # remainder in bytes of the file is incomplete or corrupt
    if nrem:
        warnings.warn(f'There is not a whole number of records in the file. nrec={nrec:d}, remainder={nrem:d} bytes.')
    return nrec, nrem

In [116]:
wd = '/Users/Theo/GRWMODELS/python/Nectaerra/Kruiszwin/cases/Kruiszwin_1/'
fname = 'justmodflow.hds'

# The binary file is read into a recarray

See the dtype for this array.

Only the complete records in the file are read. This also works with a corrupt or incomplete
binary file. So the data that can be read can still be used.


In [165]:
fp = open(os.path.join(wd, fname), 'rb')

fmt, precision, ncol, nrow = get_binary_fmt(fp) # get the correct format (by determining the file precision)

nrec, nrem = recs_in_file(fp, fmt) # get the number of complete records and see of there are any left-over bytes

print(f"The file's float precision = '{precision:s}'. Ncol={ncol:d}, Nrow={nrow:d}.") 
print(f"The number of complete rectords in the file is {nrec:d} with {nrem:d} remaining bytes.")
print("The file length is obtained from the file pointer and from the number of records\m")
print("with their length and left-over bytes. They should be the same.")
print("File lenght: {} and {}.".format(filelen(fp), nrec * struct.calcsize(fmt) + nrem))

# Define the dtype for the array to store all data per layer, kstp, kper as complete records
dtype = np.dtype([('kstp', '<i4'), ('kper', '<i4'),
                  ('pertim', precision), ('totim', precision),
                  ('desc', 'U', 16),
                  ('ncol', '<i4'), ('nrow', '<i4'), ('ilay', '<i4'), ('values', precision, (nrow, ncol))])

record = np.zeros(nrec, dtype=dtype) # Initialize the spaceto allow filling storing the data in a structured way.

buflen = struct.calcsize(fmt)

fp.seek(0, 0) # Not needed, but just to make sure

# Fill the container, record by record (and item by item for each record)
for i in range(nrec):
    buff = fp.read(buflen)

    kstp, kper, pertim, totim, desc, ncol, nrow, ilay, values = struct.unpack(fmt, buff)

    r = record[i]
    r['kstp'  ] = kstp
    r['kper'  ] = kper
    r['pertim'] = pertim
    r['totim' ] = totim
    r['desc'  ] = desc.decode('utf-8')
    r['ncol'  ] = ncol
    r['nrow'  ] = nrow
    r['ilay'  ] = ilay
    r['values'] = np.frombuffer(values, precision).reshape((nrow, ncol))  
fp.close()

# Print the last record
print("The last record:")
record[-1]

The file's float precision = '<f4'. Ncol=160, Nrow=1.
The number of complete rectords in the file is 71 with 588 remaining bytes.
The file length is obtained from the file pointer and from the number of records\m
with their length and left-over bytes. They should be the same.
File lenght: 49152 and 49152.
The last record:


/var/folders/90/m51x_b713y561gzh2kzy18d00000gq/T/ipykernel_30992/2438178817.py:54: UserWarning: There is not a whole number of records in the file. nrec=71, remainder=588 bytes.
  warnings.warn(f'There is not a whole number of records in the file. nrec={nrec:d}, remainder={nrem:d} bytes.')


(1, 1, 180., 180., '            HEAD', 160, 1, 71, [[-0.68      , -0.6799832 , -0.67996764, -0.6799534 , -0.67994034, -0.67992854, -0.6799179 , -0.6799083 , -0.67989963, -0.679892  , -0.67988515, -0.67987907, -0.67987365, -0.6798689 , -0.67986465, -0.6798609 , -0.6798576 , -0.6798547 , -0.6798521 , -0.67984986, -0.6798479 , -0.67984617, -0.6798446 , -0.67984325, -0.6798421 , -0.67984104, -0.67984015, -0.6798394 , -0.67983866, -0.679838  , -0.67983747, -0.67983705, -0.67983663, -0.6798363 , -0.6798359 , -0.6798357 , -0.6798354 , -0.6798352 , -0.679835  , -0.67983484, -0.67983466, -0.67983454, -0.6798345 , -0.67983437, -0.67983425, -0.6798342 , -0.6798341 , -0.67983407, -0.679834  , -0.679834  , -0.67983395, -0.6798339 , -0.6798339 , -0.6798338 , -0.6798338 , -0.6798338 , -0.67983377, -0.67983377, -0.67983377, -0.67983377, -0.67983377, -0.67983377, -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 , -0.6798337 

In [167]:
# With only one stress period we can construct the remaining array:

record['values'].shape

(71, 1, 160)